# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2021-09-28 07:59:44,571 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

<b>Joining citations table and patents tables on cited number</b>

In [8]:
citedPatents = citations.join(patents, citations.CITED == patents.PATENT)

In [9]:
citedPatents.show(5)

+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| CITING|  CITED| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3951073|3071083|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|4401034|3071083|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|4802

<b>Changing the POSTATE of citedPatents to POSTATE1 to help in differentiating further down the line</b>

In [10]:
citedPatents1 = citedPatents.select("CITED","POSTATE","CITING")

In [11]:
citedPatents1 = citedPatents1.toDF("CITED", "POSTATE1", "CITING")

In [12]:
citedPatents1.show(10)

+-------+--------+-------+
|  CITED|POSTATE1| CITING|
+-------+--------+-------+
|3071083|    null|3951073|
|3071083|    null|4401034|
|3071083|    null|4802416|
|3071452|    null|4045189|
|3071452|    null|4192656|
|3071753|      MN|3949375|
|3071753|      MN|4120573|
|3071753|      MN|4271479|
|3071753|      MN|4345315|
|3071753|      MN|4496943|
+-------+--------+-------+
only showing top 10 rows



<b>Joining citedPatents with patents table on matching with citings</b>

In [13]:
citingPatents = citedPatents1.join(patents, citedPatents1.CITING == patents.PATENT)

In [14]:
citingPatents.show(10)

2021-09-28 08:02:52,150 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|  CITED|POSTATE1| CITING| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+--------+-------+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3638586|      CA|3858527|3858527| 1975| 5485|   1973|     CH|   null|  336690|      3|    10|   108|  6|    69|    7|       1|  0.4286|    0.0|     0.0|     2.0| 19.5714|     0.0|     0.0|     0.0|     0.0|
|3699902|      OH|3858527|3858527| 1975| 5485|   1973|     CH|   null|  336690|      3|    10|   108|  6|    69|    7|       1|  0.4286|    0.0|     0.0|     2.0| 19.57

<b>Selecting only required columns from the above table</b>

In [15]:
citingPatents = citingPatents.select("CITED","POSTATE1","CITING","POSTATE")

In [16]:
citingPatents.show(10)

+-------+--------+-------+-------+
|  CITED|POSTATE1| CITING|POSTATE|
+-------+--------+-------+-------+
|3638586|      CA|3858527|   null|
|3699902|      OH|3858527|   null|
|3602157|      TX|3858527|   null|
|3815160|      NY|3858597|     MT|
|3675252|      AZ|3858597|     MT|
|3741706|      OH|3859029|     NY|
|3685936|      OH|3859029|     NY|
|3368197|      MI|3859627|     MI|
|3226674|    null|3859627|     MI|
|3359539|      CA|3859627|     MI|
+-------+--------+-------+-------+
only showing top 10 rows



<b>Filtering only the citings/citeds where state matches and is not equal to null</b>

In [17]:
citingPatents1 = citingPatents.filter((citingPatents.POSTATE1 == citingPatents.POSTATE) & (citingPatents.POSTATE1 != 'null') & (citingPatents.POSTATE != 'null'))

In [18]:
citingPatents1.show(10)

+-------+--------+-------+-------+
|  CITED|POSTATE1| CITING|POSTATE|
+-------+--------+-------+-------+
|3464385|      AK|4178878|     AK|
|3745074|      AL|3974004|     AL|
|3585090|      AL|3974004|     AL|
|3762972|      AL|3974004|     AL|
|3692600|      AL|3974004|     AL|
|3373564|      AL|4554823|     AL|
|3972467|      AL|4698246|     AL|
|3972467|      AL|4701360|     AL|
|5026073|      AL|5078406|     AL|
|5623808|      AL|5701722|     AL|
+-------+--------+-------+-------+
only showing top 10 rows



<b>Grouping on citings and couting the number of same state citations. Also sorting in descending order<b>

In [19]:
citingPatents2 = citingPatents1.groupBy("CITING").count().sort("count", ascending=False)

In [20]:
citingPatents2.show(15)

+-------+-----+
| CITING|count|
+-------+-----+
|5959466|  125|
|5983822|  103|
|6008204|  100|
|5952345|   98|
|5998655|   96|
|5958954|   96|
|5936426|   94|
|5913855|   90|
|5980517|   90|
|5978329|   90|
|5739256|   90|
|5925042|   90|
|5951547|   90|
|5618907|   89|
|5917082|   89|
+-------+-----+
only showing top 15 rows



<b>Joining citingPatents2 table with original patents table to show the final output</b>

In [21]:
output = patents.join(citingPatents2, patents.PATENT == citingPatents2.CITING, "left_outer")
output.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|CITING|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+------+-----+
|3070853| 1963| 1096|   null|     US|     FL|    null|      1|  null|    49|  5|    59| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|  null| null|
|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|  null| null|
|3071452| 1963| 1096

In [22]:
output.count()

2923922

In [23]:
finalOutput = output.sort("count", ascending=False)

In [24]:
finalOutput1 = finalOutput.select("PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD","count")

In [25]:
finalOutput1 = finalOutput1.toDF("PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD","SAME_STATE")

In [26]:
finalOutput1.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|       103|
|6008204| 1999|14606|   1998| 